In [76]:
import seaborn as sns
import pandas as pd
from ads_surkas.preprocessing import load_data, clean_data
import numpy as np
import math

In [77]:
# load data
# data = pd.read_csv("./.data/pakistan_ecommerce.csv", low_memory=False)
# data.loc[1048570]
data = clean_data(load_data("./.data/pakistan_ecommerce.csv"))

# data["Year"] = data["Year"].astype("int32")

# data["Month"] = data["Month"].astype("int32")
# print(data.dtypes)
#data["Time"] = pd.to_datetime(data["Year"]+data["Month"]+"1",format='%Y-%m-%d')
# data[(data["Year"] != 2016.0) & (data["Year"] != 2017.0) & (data["Year"] != 2018.0)]["Year"]
#data[["grand_total", "status"]].sort_values("grand_total").head(20)
data.head(20)


/home/ulrik2204/code/school/ads_surkas/ads_surkas/preprocessing/preprocess.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["Year"] = data["Year"].astype("int32")
/home/ulrik2204/code/school/ads_surkas/ads_surkas/preprocessing/preprocess.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["Month"] = data["Month"].astype("int32")
/home/ulrik2204/code/school/ads_surkas/ads_surkas/preprocessing/preprocess.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Dat

,item_id,status,created_at,sku,Customer ID,increment_id,category_name_1,sales_commission_code,discount_amount,payment_method,...,BI Status,Customer Since,M-Y,FY,price,qty_ordered,grand_total,Time,Year,Month
0,211131.0,complete,7/1/2016,kreations_YI 06-L,1.0,100147443,Women's Fashion,\N,0.0,cod,...,#REF!,2016-07-01,7-2016,FY17,1950.00,1.0,1950.00,2016-07-01,2016,7
1,211133.0,canceled,7/1/2016,kcc_Buy 2 Frey Air Freshener & Get 1 Kasual Bo...,2.0,100147444,Beauty & Grooming,\N,0.0,cod,...,Gross,2016-07-01,7-2016,FY17,240.00,1.0,240.00,2016-07-01,2016,7
2,211134.0,canceled,7/1/2016,Ego_UP0017-999-MR0,3.0,100147445,Women's Fashion,\N,0.0,cod,...,Gross,2016-07-01,7-2016,FY17,2450.00,1.0,2450.00,2016-07-01,2016,7
3,211135.0,complete,7/1/2016,kcc_krone deal,4.0,100147446,Beauty & Grooming,R-FSD-52352,300.0,cod,...,Net,2016-07-01,7-2016,FY17,360.00,1.0,60.00,2016-07-01,2016,7
4,211136.0,order_refunded,7/1/2016,BK7010400AG,5.0,100147447,Soghaat,\N,0.0,cod,...,Valid,2016-07-01,7-2016,FY17,555.00,2.0,1110.00,2016-07-01,2016,7
5,211137.0,canceled,7/1/2016,UK_Namkino All In One 200 Gms,6.0,100147448,Soghaat,\N,0.0,cod,...,Gross,2016-07-01,7-2016,FY17,80.00,1.0,80.00,2016-07-01,2016,7
6,211138.0,complete,7/1/2016,kcc_krone deal,7.0,100147449,Beauty & Grooming,\N,300.0,cod,...,Net,2016-07-01,7-2016,FY17,360.00,1.0,60.00,2016-07-01,2016,7
7,211139.0,complete,7/1/2016,UK_Namkino Mix Nimco 400 Gms,6.0,100147450,Soghaat,\N,0.0,cod,...,Net,2016-07-01,7-2016,FY17,170.00,1.0,170.00,2016-07-01,2016,7
8,211140.0,canceled,7/1/2016,Apple iPhone 6S 64GB,8.0,100147451,Mobiles & Tablets,\N,0.0,ublcreditcard,...,Gross,2016-07-01,7-2016,FY17,96499.00,1.0,96499.00,2016-07-01,2016,7
9,211141.0,canceled,7/1/2016,Apple iPhone 6S 64GB,8.0,100147452,Mobiles & Tablets,\N,0.0,mygateway,...,Gross,2016-07-01,7-2016,FY17,96499.00,1.0,96499.00,2016-07-01,2016,7


In [78]:
END_DATE = pd.to_datetime("2018-08-18")
ROWS = 115326
TILE = 28831


def label_customer(row, column, thresholds, labels):
    for i, th in enumerate(thresholds):
        min_th = -math.inf if i == 0 else thresholds[i - 1]
        if min_th < row[column] <= th:
            return labels[i]
    return labels[-1]


def label_customer_all_time(row):
    thresholds = [7500, 39717, 114671.58, 265709.29, 575067.925, 1323656]
    labels = [
        "tiny",
        "small",
        "medium_small",
        "medium",
        "medium_large",
        "large",
        "very_large",
    ]
    return label_customer(row, "all_time_total", thresholds, labels)




customer_groups_all_time = (
    data.groupby("increment_id")
    .first()
    .groupby("Customer ID", as_index=False)
    .agg(
        all_time_total=("grand_total", "sum"),
        order_count=("grand_total", "count"),
        # customer_since=("Customer Since", "first"),
    )
)
# customer_groups_all_time["all_time_total_per_month"] = round(
#     customer_groups_all_time["all_time_total"]
#     / (
#         (END_DATE - customer_groups_all_time["customer_since"]) / np.timedelta64(1, "D")
#     ).astype(int)
#     * 30,
#     2,
# )

# customer_groups_all_time.sort_values("all_time_total").iloc[
#     math.floor(ROWS / 4) * 2 : math.floor(ROWS / 4 * 3)
# ]
customer_groups_all_time["group"] = customer_groups_all_time.apply(
    label_customer_all_time, axis=1
)
# customer_groups_all_time["group_month"] = customer_groups_all_time.apply(
#     label_customer_per_month, axis=1
# )
# a print(customer_groups_all_time)
# print(customer_groups_all_time.groupby("group")["Customer ID"].count())
# customer_groups_all_time.groupby("group")["all_time_total"].sum().reset_index().plot.bar(x="group")
# dat.to_csv(".data/all_time_total.csv")

# Finding groups of customers based on all_time_total
# dat = (
#     data.groupby("increment_id")
#     .first()
#     .groupby("customer id", as_index=false)
#     .agg(all_time_total=("grand_total", "sum"), order_count=("grand_total", "count"))
# )
# print(dat.quantile(0.63)) #  7500.00
# print(dat.quantile(0.86)) #   39717.33
# print(dat.quantile(0.95)) #  114671.5875
# print(dat.quantile(0.98)) #  265709.29
# print(dat.quantile(0.993)) # 575067.925
# print(dat.quantile(0.9975)) #  1323656

# print("Here starts all_time_total_per_month....")
# dat = customer_groups_all_time.groupby("Customer ID", as_index=False)[
#     "all_time_total_per_month"
# ].first()
# print(dat.quantile(0.63))  #  7500.00
# print(dat.quantile(0.86))  #   39717.33
# print(dat.quantile(0.95))  #  114671.5875
# print(dat.quantile(0.98))  #  265709.29
# print(dat.quantile(0.993))  # 575067.925
# print(dat.quantile(0.9975))  #  1323656
GROUPS = 4
TILE = 0
# customer_groups_all_time.sort_values("all_time_total_per_month").iloc[
#     math.floor(ROWS / GROUPS) * TILE : math.floor((ROWS / GROUPS) * (TILE + 1))
# ]
# customer_groups_all_time.groupby("group_month")[
#     "Customer ID"
# ].count()  # .plot.bar(x="group_month")
# customer_groups_all_time.groupby("group_month")[
#     "all_time_total"
# ].sum().reset_index().plot.bar(x="group_month")
customer_groups_all_time

,Customer ID,all_time_total,order_count,group
0,1.0,1950.00,1,tiny
1,2.0,510.00,2,tiny
2,3.0,3695.00,2,tiny
3,4.0,2493024.45,387,very_large
4,5.0,1110.00,1,tiny
...,...,...,...,...
115321,115322.0,2596.00,2,tiny
115322,115323.0,44499.00,1,medium_small
115323,115324.0,549.00,1,tiny
115324,115325.0,1199.00,2,tiny


In [79]:
customer_total_per_month = (
    data.groupby(["increment_id"]).first().groupby(["Customer ID", "Time"], as_index=False).agg(monthly_total=("grand_total", "sum"))
    # data.groupby("Time", as_index=False).agg(monthly_total=("grand_total", "sum"), increment_id=("increment_id", "first"))
    # .groupby("Customer ID", as_index=False).agg(monthly_total=("grand_total", "sum"))
)
# customer_total_per_month.plot.bar()  #[["Customer ID", "Time", "grand_total"]]

def label_customer_per_month(row):
    # thresholds = [95.95, 342.365, 1588]
    thresholds = [705.88, 4235.29, 14863.215, 36291.42, 87042.101, 205092.096875]
    labels = [
        "tiny_monthly",
        "small_monthly",
        "medium_small_monthly",
        "medium_monthly",
        "medium_large_monthly",
        "large_monthly",
        "very_large_monthly",
    ]
    return label_customer(row, "monthly_total", thresholds, labels)

customer_total_per_month["group_monthly"] = customer_total_per_month.apply(label_customer_per_month, axis=1)
# data.groupby("Customer ID").first().count()
# customer_total_per_month[customer_total_per_month["Customer ID"] == 5032]
# customer_total_per_month.groupby("Time", as_index=False).agg(group_count=("group_monthly", "count")).line(x="Time", y="group_count")
# customer_total_per_month.groupby("group_monthly")["Customer ID"].count().plot.pie()

In [80]:
# Add the customer categories to the data table
new_column = customer_groups_all_time.merge(customer_total_per_month, on="Customer ID")
data = pd.merge(data, new_column, on=["Customer ID", "Time"], how="left")
data.to_csv("./.data/pakistan_ecommerce_with_categories.csv")
# data[data["Customer ID"] == 5032] #[["Customer ID", "grand_total", "all_time_total", "monthly_total" "Customer Since"]]
# new_column[new_column["Customer ID"] == 5032]
# new_column
#print(data[["group"]])
data.groupby("Customer ID", as_index=False).first()# .groupby( "group", as_index=False).agg(count_of_distinct_customers=("all_time_total", "sum")).plot.bar( x="group", y="count_of_distinct_customers")  # .reset_index()["count_of_distinct_customers"].sum() #plot.bar(x="group_monthly", y="count_of_distinct_customers")
data.dtypes
new_column


,Customer ID,all_time_total,order_count,group,Time,monthly_total,group_monthly
0,1.0,1950.0,1,tiny,2016-07-01,1950.0,small_monthly
1,2.0,510.0,2,tiny,2016-07-01,240.0,tiny_monthly
2,2.0,510.0,2,tiny,2016-11-01,270.0,tiny_monthly
3,3.0,3695.0,2,tiny,2016-07-01,2450.0,small_monthly
4,3.0,3695.0,2,tiny,2017-05-01,1245.0,small_monthly
...,...,...,...,...,...,...,...
169937,115322.0,2596.0,2,tiny,2018-08-01,2596.0,small_monthly
169938,115323.0,44499.0,1,medium_small,2018-08-01,44499.0,medium_large_monthly
169939,115324.0,549.0,1,tiny,2018-08-01,549.0,tiny_monthly
169940,115325.0,1199.0,2,tiny,2018-08-01,1199.0,small_monthly


In [81]:
# Categorize customers per all_time_total/customer_since
# customer_groups_per_month = customer_groups_all_time.copy()
# data["all_time_total_per_month"] = round(data["all_time_total"] / ((END_DATE - data["Customer Since"]) / np.timedelta64(1, "D")).astype(int) * 30, 2)
# data[["Customer ID", "all_time_total", "group", "increment_id", "Customer Since", "all_time_total_per_month"]]
# data.sort_values("all_time_total_per_month").groupby("increment_id").first().groupby("Customer ID", as_index=False).agg()
# data

In [82]:
data

,item_id,status,created_at,sku,Customer ID,increment_id,category_name_1,sales_commission_code,discount_amount,payment_method,...,qty_ordered,grand_total,Time,Year,Month,all_time_total,order_count,group,monthly_total,group_monthly
0,211131.0,complete,7/1/2016,kreations_YI 06-L,1.0,100147443,Women's Fashion,\N,0.0,cod,...,1.0,1950.0,2016-07-01,2016,7,1950.00,1.0,tiny,1950.0,small_monthly
1,211133.0,canceled,7/1/2016,kcc_Buy 2 Frey Air Freshener & Get 1 Kasual Bo...,2.0,100147444,Beauty & Grooming,\N,0.0,cod,...,1.0,240.0,2016-07-01,2016,7,510.00,2.0,tiny,240.0,tiny_monthly
2,211134.0,canceled,7/1/2016,Ego_UP0017-999-MR0,3.0,100147445,Women's Fashion,\N,0.0,cod,...,1.0,2450.0,2016-07-01,2016,7,3695.00,2.0,tiny,2450.0,small_monthly
3,211135.0,complete,7/1/2016,kcc_krone deal,4.0,100147446,Beauty & Grooming,R-FSD-52352,300.0,cod,...,1.0,60.0,2016-07-01,2016,7,2493024.45,387.0,very_large,50155.0,medium_large_monthly
4,211136.0,order_refunded,7/1/2016,BK7010400AG,5.0,100147447,Soghaat,\N,0.0,cod,...,2.0,1110.0,2016-07-01,2016,7,1110.00,1.0,tiny,1110.0,small_monthly
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
584492,905204.0,cod,8/28/2018,WOFSCE5AE00357AECDE,115320.0,100562385,Women's Fashion,NaN,0.0,cod,...,1.0,849.0,2018-08-01,2018,8,2596.50,2.0,tiny,2596.5,small_monthly
584493,905205.0,processing,8/28/2018,MATHUA5AF70A7D1E50A,115326.0,100562386,Mobiles & Tablets,NaN,0.0,bankalfalah,...,1.0,35899.0,2018-08-01,2018,8,35899.00,1.0,small,35899.0,medium_monthly
584494,905206.0,processing,8/28/2018,MATSAM5B6D7208C6D30,113474.0,100562387,Mobiles & Tablets,NaN,0.0,bankalfalah,...,2.0,652178.0,2018-08-01,2018,8,5676115.00,35.0,very_large,5165122.0,very_large_monthly
584495,905207.0,processing,8/28/2018,MATSAM5B1509B4696EA,113474.0,100562387,Mobiles & Tablets,NaN,0.0,bankalfalah,...,2.0,652178.0,2018-08-01,2018,8,5676115.00,35.0,very_large,5165122.0,very_large_monthly


In [83]:
# Orders per time only (and their grand total)
data["original_price"] = data["price"] * data["qty_ordered"]
data["discount_percentage"] = data["discount_amount"] / data["original_price"]
data.groupby("increment_id", as_index=False).agg(
    grand_total=("grand_total", "first"),
    Time=("Time", "first"),
    customer_id=("Customer ID", "first"),
    group=("group", "first"),
    group_monthly=("group_monthly", "first"),
    monthly_total=("monthly_total", "first"),
    all_time_total=("all_time_total", "first"),
    discount_amount=("discount_amount", "sum"),
    status=("status", "first"),
    qty_ordered=("qty_ordered", "sum"),
    customer_since=("Customer Since", "first"),
    payement_method=("payment_method", "first"),
    original_price=("original_price", "sum"),
    mean_discount_percentage=("discount_percentage", "mean")

).to_csv(".data/orders_per_time.csv")
# data[["discount_amount"]]
data.head(20)


,item_id,status,created_at,sku,Customer ID,increment_id,category_name_1,sales_commission_code,discount_amount,payment_method,...,Time,Year,Month,all_time_total,order_count,group,monthly_total,group_monthly,original_price,discount_percentage
0,211131.0,complete,7/1/2016,kreations_YI 06-L,1.0,100147443,Women's Fashion,\N,0.0,cod,...,2016-07-01,2016,7,1950.00,1.0,tiny,1950.00,small_monthly,1950.00,0.000000
1,211133.0,canceled,7/1/2016,kcc_Buy 2 Frey Air Freshener & Get 1 Kasual Bo...,2.0,100147444,Beauty & Grooming,\N,0.0,cod,...,2016-07-01,2016,7,510.00,2.0,tiny,240.00,tiny_monthly,240.00,0.000000
2,211134.0,canceled,7/1/2016,Ego_UP0017-999-MR0,3.0,100147445,Women's Fashion,\N,0.0,cod,...,2016-07-01,2016,7,3695.00,2.0,tiny,2450.00,small_monthly,2450.00,0.000000
3,211135.0,complete,7/1/2016,kcc_krone deal,4.0,100147446,Beauty & Grooming,R-FSD-52352,300.0,cod,...,2016-07-01,2016,7,2493024.45,387.0,very_large,50155.00,medium_large_monthly,360.00,0.833333
4,211136.0,order_refunded,7/1/2016,BK7010400AG,5.0,100147447,Soghaat,\N,0.0,cod,...,2016-07-01,2016,7,1110.00,1.0,tiny,1110.00,small_monthly,1110.00,0.000000
5,211137.0,canceled,7/1/2016,UK_Namkino All In One 200 Gms,6.0,100147448,Soghaat,\N,0.0,cod,...,2016-07-01,2016,7,250.00,2.0,tiny,250.00,tiny_monthly,80.00,0.000000
6,211138.0,complete,7/1/2016,kcc_krone deal,7.0,100147449,Beauty & Grooming,\N,300.0,cod,...,2016-07-01,2016,7,105.00,4.0,tiny,60.00,tiny_monthly,360.00,0.833333
7,211139.0,complete,7/1/2016,UK_Namkino Mix Nimco 400 Gms,6.0,100147450,Soghaat,\N,0.0,cod,...,2016-07-01,2016,7,250.00,2.0,tiny,250.00,tiny_monthly,170.00,0.000000
8,211140.0,canceled,7/1/2016,Apple iPhone 6S 64GB,8.0,100147451,Mobiles & Tablets,\N,0.0,ublcreditcard,...,2016-07-01,2016,7,192998.00,2.0,medium,192998.00,large_monthly,96499.00,0.000000
9,211141.0,canceled,7/1/2016,Apple iPhone 6S 64GB,8.0,100147452,Mobiles & Tablets,\N,0.0,mygateway,...,2016-07-01,2016,7,192998.00,2.0,medium,192998.00,large_monthly,96499.00,0.000000


In [84]:
# # Total Revenue in November 2016
# total = 0
# for i, row in data.groupby("increment_id").first().reset_index().iterrows():
#     order_grand_total = row["grand_total"]
#     y = row["Time"].year
#     m = row["Time"].month
#     if (m == 11 and y == 2016):
#         total += order_grand_total
# print("total: ", total)

In [85]:
# total1 = 0
# total2 = 0
# total3 = 0
# checked = []
# for i, row in data.iterrows():
#     if row["Customer ID"] == 5032 and row["increment_id"] not in checked:
#         total2 += row["grand_total"]
#         checked.append(row["increment_id"])
#     if row["Customer ID"] == 5032: 
#         total1 += row["price"] * row["qty_ordered"]
#         total3 += row["price"] * row["qty_ordered"] - row["discount_amount"]
#         checked.append(row["increment_id"])

In [86]:
# print("price * qty", total1)
# print("sum of grand total", total2)
# print("price * qty - discount", total3)

In [87]:
# dat = data.groupby("increment_id").first().groupby("Customer ID", as_index=False).agg(all_time_total=("grand_total","sum"), order_count=("grand_total", "count"))
# dat = data.copy() #.groupby("Customer ID", as_index=False).agg(all_time_total=("grand_total","sum"), order_count=("grand_total", "count"))
# dat["all_time_price"] = dat["price"] * dat["qty_ordered"] - dat["discount_amount"]
# dat = dat.filter(["Customer ID", "increment_id", "price", "qty_ordered", "discount_amount", "grand_total", "all_time_price"])
# cust = dat.groupby("Customer ID", as_index=False).agg(total_price=("all_time_price", "sum"))
# all = cust.merge(dat, on="Customer ID")
# dat[dat["Customer ID"] == 5032]

# dat[dat["Customer ID"] == 5032].tail(20)

In [88]:
# g = sns.PairGrid(data, vars=['category_name_1', 'grand_total', 'Time'])
# g.map_upper(sns.histplot)